### CSE317L-3 Midterm Lab Assignment: K-Means clustering on MNIST data
#### 1910456 Mir Shafayat Ahmed

![MNIST Images](MNISTImages.png)

We will in this assignment cluster images from MNIST (Mixed National Institute of Standards) database of handwritten digits. MNIST data set containing N = 60000 grayscale images of size 28 × 28 as shown above. We will follow the steps specified below:

**Load data:** Download the file `mnist_train.mat` from [here](https://drive.google.com/file/d/1xKu-62Xq49V27f_LWi4Qu1QzlpXkb9K4/view?usp=sharing) and load it into two variables in your python program. Use the appropriate loading function for the `.mat` file.
This creates two variables: a 784 × 60000 matrix `digits` and a 1 × 60000 matrix `labels`.
We will not need labels. 
__Each column of digits is a 28 × 28 grayscale image, stored as a vector of length $28^2 = 784$ with elements between 0 and 1 (0 denotes a black pixel and 1 a white pixel).__

**Display data:** To display the image in the $i^{th}$ column of digits you can `reshape` the column into a 28 by 28 matrix.
Then you can show it.
To speed up the computations we will use only the first 10000 digits. That means the first 10000 columns. The pseudo-code is:
`digits = digits(:, 1:10000);`

**Apply K-means:** You are asked to apply the k-means algorithm to this set of __N = 10000 vectors, with k = 10 groups, and starting from a random initial group assignment__

**Algorithm 4.1** k-means algorithm
**given** a list of N vectors $x_1,...,x_N$, and an initial list of $k$ group representative vectors (centroids) $z_1,...,z_k$ repeat until convergence.

1. _Partition_ the vectors into k groups. For each vector $i = 1,...,N$, assign $x_i$ to the group associated with the nearest centroids.
1. _Update representatives (centroids)_. For each group $j = 1,...,k$, set $z_j$ to be the mean of the vectors in group $j$.
- We evaluate the quality of the clustering using the clustering objective
$$
J = \frac{1}{N}\sum^N_{i=1}\min_{j=1,...,k}\|x_i - z_j\|^2
$$
The algorithm is terminated when J is nearly equal in two successive iterations (e.g., we terminate when $|J − J_{prev}| ≤ 10^{−5}J$, where $J_{prev}$ is the value of J after the previous iteration).
- After running the k-means algorithm you can display the representative vectors(centroid vectors) of the 10
groups in subplots by running a loop

In [ ]:
# NOTE: COMMENT OUT THE LAST CELL OF THE NOTEBOOK IF USING LOCALMACHINE

In [ ]:
# importing scipy to load the MATLAB file
from scipy.io import loadmat
import numpy as np
import pandas as pd
from pathlib import Path
from os import remove, walk, listdir
from PIL import Image as im, ImageEnhance
import imageio
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import Image

# NOTE: Uncomment 1 option and keep others commented

# ################################### DIRECT DOWNLOAD FROM GOOGLE DRIVE SOURCE ########################################

!pip install --upgrade --no-cache-dir gdown

file_share_link = "https://drive.google.com/file/d/1xKu-62Xq49V27f_LWi4Qu1QzlpXkb9K4/view?usp=sharing"
file_id = file_share_link[file_share_link.find("/d/") + 3: file_share_link.find("/view")]
print(file_id)

!gdown --id $file_id
matrix = loadmat('mnist_train.mat')

#OR-----------------------------

# ################################### IMPORTING FROM PERSONAL GOOGLE DRIVE ###################################

# from google.colab import drive
# drive.mount('/content/drive')

# matrix = loadmat('/content/drive/MyDrive/Classroom/Spring-2022-CSE317-3 CSC317,CEN317/1910456 Mir Shafayat Ahmed Lab Midterm Assignment/mnist_train.mat')

#OR-----------------------------

# ################################### Uploading the mnist_train.mat file in colab manually ###################################

# matrix = loadmat('mnist_train.mat')

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=fbc65eec20cd48e5f018a2e9f8a49aed7aad6d59e3b5e02f1b2760e18e06f55a
  Stored in directory: /tmp/pip-ephem-wheel-cache-_4b8v0z_/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2
1xKu-62Xq49V27f_LWi4Qu1QzlpXkb9K4
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1xKu-62Xq49V27f_LWi4Qu1QzlpXkb9K4
To: /content/mnist_train.mat
100% 26.5M/26.5M [00:00<00:00, 106MB/s] 


In [ ]:
subset = 20000
k = 15          # Deciding how many Centroids

In [ ]:
datas = np.transpose(matrix['digits'])  # So that each element is a 784 dimentional vector datapoint
datas = datas[:subset]                # Taking only the first few of the total
labels = matrix['labels'][0]            # Getting the labels
labels = labels[:subset]
row, col = np.shape(datas)

print(f'Shape: {row} rows by {col} columns')

pixelColumns = [ 'pixel'+str(i) for i in range(datas.shape[1]) ]   # Creating column headers for each element of the pixel color vector
df = pd.DataFrame(datas,columns=pixelColumns)                      # Creating a dataframe that would hold the data.
df['label'] = labels                                            # Creating a column named "label" for later use in coloring/labeling the datapoints

datas, labels = None, None                                      # No longer needed

print(df)

Shape: 20000 rows by 784 columns
       pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
19995     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
19996     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
19997     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
19998     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
19999     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.

In [ ]:
centroids = np.zeros((k, col))          # Initializing the k number of Centroids
data_subset = df[pixelColumns].to_numpy()  # creating a numpy array with only the vectors

randidx = np.random.permutation(row)    # Selecting random data points for each initial Centroid
for i in range(k):
    centroids[i] = data_subset[randidx[i]]

group = np.zeros(row)-1                 # Creating an (essentially) empty array to keep the group number that corresponds to each datapoint.


In [ ]:
j_prev = np.inf                         # So that loop does not break immediately after first Iteration.
for i in range(150):                    # Setting a max limit of loops so that it is not infinite.

    for dp_idx, dp in enumerate(data_subset):              # getting both index and value(vector) of a single datapoint from the set (data)
        minDist = np.inf                            # So that at least one datapoint is considered to be close to a centroid.
        for ct_idx, ct in enumerate(centroids):     # Getting both index and value(vector) of a single centroid from the set (centroids)
            sq_dist = np.dot(dp-ct, dp-ct)          # ||datapoint - centroid||^2 or "Norm Square" or the "Squared Distance" between the two vectors
            if sq_dist < minDist:                   
                minDist = sq_dist                   # Setting the current minimum distance
                group[dp_idx] = ct_idx              # Setting the current centroid (index) which seems closest to the current datapoint (index)
    for ct_idx in range(k):                         # Iterating over all centroid indexes
        groupedDps = data_subset[group == ct_idx]          # The actual datapoints closest to the current centroid are extracted by using a boolean array 
        oneVect = np.ones(len(groupedDps))
        centroids[ct_idx] = (np.dot(oneVect, groupedDps)/len(groupedDps))   # The new centroid is now the mean of the grouped datapoints 
        # centroids[ct_idx] = np.mean(groupedDps, axis=0)


    # The next bit of code till `img.save(centroidImgPath)` is for generating png images of the centroids. Thus not related to the assignment
    
    centroidCopy = centroids                        # Creating a copy so that the set of Centroids are not modified.
    for idx, centroid in enumerate(centroidCopy):   # getting both index and value(vector) of a single centroid from the set (centroidCopy)
        centroidsFolderPath = Path('CENTROIDS')     # Setting Folder name where each centroid would be stored (as images)
        centroidSavePath = centroidsFolderPath.joinpath(f'centroid {str(idx)}')     # Setting specific folders for each centroid, e.g centroid 0, centroid 5

        if not Path.exists(centroidsFolderPath):
            Path.mkdir(centroidsFolderPath)
        if not Path.exists(centroidSavePath):
            Path.mkdir(centroidSavePath)
        
        centroid = np.reshape(centroid , (28,28)) * 90  # Reshaping the 1D, 784dim Vectors into a 28 by 28  Matrix and scaling ...
                                                        # ... each element by 90 to increase brightness
        centroid = np.transpose(centroid)               # Transposing so that the orientation of the generated image is correct and not flipped/rotated  .  
        img = im.fromarray(centroid)                    # Creating the immage from the centroid matrix.
        img = img.convert('L')                          # Mode "L" means image has 8-bit pixels, black and white.
        enhancer = ImageEnhance.Contrast(img)           # For increasing contrast
        img = enhancer.enhance(3.5)                     # By 3.5 times

        centroidImgPath = centroidSavePath.joinpath(f'{i}.png')     # The current iteration number is the name of the image to be saved
        if Path.exists(centroidImgPath):
            for f in listdir(centroidSavePath):
                remove(centroidSavePath.joinpath(f))    # Remove all images in the folder if one already exists (because they are probably from previous runs)
        img.save(centroidImgPath)


    # The Clustering Objective
    sum = 0                     # Initial sum is 0
    for x_i in data_subset:            # Iterating through each datapoint
        sum += min([np.dot(x_i - z_j, x_i - z_j) for z_j in centroids])     # List comprehension to get a list of all the squared distances between...
                                                                            # ...the current datapoint and each of the k centroids. Then using min()...
                                                                            # ...to get the minimum distance and adding it to sum.
    j = sum/row                                                             # J is found by dividing sum with the number of datapoints. 1/N * sum

    print(f'Iteration {i}, |J-J_prev| = {abs(j-j_prev)}')
    if abs(j-j_prev) <= 10**(-5):                           # Breaks the loop if there is no significant change in the value of J over an iteration
        break
    j_prev = j                                              # Updating J_prev for later iterations.

Iteration 0, |J-J_prev| = inf
Iteration 1, |J-J_prev| = 0.8454097359398673
Iteration 2, |J-J_prev| = 0.3005138169291186
Iteration 3, |J-J_prev| = 0.16439034929330631
Iteration 4, |J-J_prev| = 0.10123071891370472
Iteration 5, |J-J_prev| = 0.06435795822826407
Iteration 6, |J-J_prev| = 0.048323999370886384
Iteration 7, |J-J_prev| = 0.039156769305982664
Iteration 8, |J-J_prev| = 0.025346836032355213
Iteration 9, |J-J_prev| = 0.016717065744145998
Iteration 10, |J-J_prev| = 0.011126457538814805
Iteration 11, |J-J_prev| = 0.00836940763916516
Iteration 12, |J-J_prev| = 0.005867382447426905
Iteration 13, |J-J_prev| = 0.004718334274535607
Iteration 14, |J-J_prev| = 0.005037078769980496
Iteration 15, |J-J_prev| = 0.0037653382289732917
Iteration 16, |J-J_prev| = 0.003019475668580185
Iteration 17, |J-J_prev| = 0.002357347191455972
Iteration 18, |J-J_prev| = 0.0019329664556835269
Iteration 19, |J-J_prev| = 0.0017782950557077015
Iteration 20, |J-J_prev| = 0.0016754470536710642
Iteration 21, |J-J_prev

In [ ]:
centroidDF = pd.DataFrame(centroids, columns = pixelColumns)       # Creating a df that will only store centroid data but has the same structure as the original df

centroidDF['label'] = ["centroid"] * k          # Setting the "label" as "centroid" for all centroids

dfWithCentroids = pd.concat([df, centroidDF])                   # making a df with the original and centroid df
dfWithCentroids_subset = dfWithCentroids[pixelColumns]

In [ ]:
tsne = TSNE(n_components=3, verbose=1, perplexity=40, n_iter=300)   # Setting up t-SNE for 3D vizualisation of data
tsne_results = tsne.fit_transform(dfWithCentroids_subset)           # applying t-SNE

dfWithCentroids['t-SNE-comp-1'] = tsne_results[:,0]          # Creating 3 columns to store the results of the three components
dfWithCentroids['t-SNE-comp-2'] = tsne_results[:,1]
dfWithCentroids['t-SNE-comp-3'] = tsne_results[:,2]

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 20015 samples in 0.019s...
[t-SNE] Computed neighbors for 20015 samples in 40.403s...
[t-SNE] Computed conditional probabilities for sample 1000 / 20015
[t-SNE] Computed conditional probabilities for sample 2000 / 20015
[t-SNE] Computed conditional probabilities for sample 3000 / 20015
[t-SNE] Computed conditional probabilities for sample 4000 / 20015
[t-SNE] Computed conditional probabilities for sample 5000 / 20015
[t-SNE] Computed conditional probabilities for sample 6000 / 20015
[t-SNE] Computed conditional probabilities for sample 7000 / 20015
[t-SNE] Computed conditional probabilities for sample 8000 / 20015
[t-SNE] Computed conditional probabilities for sample 9000 / 20015
[t-SNE] Computed conditional probabilities for sample 10000 / 20015
[t-SNE] Computed conditional probabilities for sample 11000 / 20015
[t-SNE] Computed conditional probabilities for sample 12000 / 20015
[t-SNE] Computed conditional probabilities for s

In [ ]:
prevData =  dfWithCentroids.iloc[:-k]      # Separating out the Centroid and Original data so that they can be shown properly in the graph...
centroidData = dfWithCentroids.iloc[-k:]

In [ ]:
def make_gif(imgFolder, saveFolder, name, frameDuration):
    # sorted(listdir(imgFolder), key=lambda x: int(x.split(".")[0])) returns the names of the images stored in imgFolder in sorted order by considering...
    #...only the name of the image (without the extension) which is an integer.
    # Each image is then opened into python and are part of a list of images (list comprehension)
    frames = [im.open(imgFolder.joinpath(image)) for image in sorted(listdir(imgFolder), key=lambda x: int(x.split(".")[0]))]
    # frame_one is the first image
    frame_one = frames[0]
    # Subsequent images are appended into frame_one and it is saved.
    frame_one.save(saveFolder.joinpath(f'{name}.gif'), format="GIF", append_images=frames,save_all=True, duration=frameDuration, loop=0)

for root, subdirectories, files in walk(centroidsFolderPath):   # Get a list of Subdirectories of the CENTROIDS folder. i.e ['centroid 0', 'centroid 1' ...]
    for subdirectory in subdirectories:                         # Iterating over a each folder
        imgFolder = Path(root, subdirectory)                    # Where the Images of each iterations are stored for the specific centroid
        name = f'centroid {subdirectory.split(" ")[1]}'         # The name of the gif file is the name of the centroid. e.g "centroid 5.gif"
        make_gif(imgFolder, centroidsFolderPath, name, 100)     # 100ms is the duration of one frame

In [ ]:
# Getting all the gifs from centroidsFolderPath.
gifs = [imageio.get_reader(Path(centroidsFolderPath, gif)) for gif in listdir(centroidsFolderPath) if gif.endswith(".gif")]
# Minimum of the number of frames from the gifs are used to avoid potential IndexError
number_of_frames = min(map(lambda gif: gif.get_length(), gifs)) 
# Writer object where all the Gifs will be merged
finalGIF = imageio.get_writer(centroidsFolderPath.joinpath('Final Centroids.gif'))

for frameNum in range(number_of_frames):
    # Each gif object is mapped to gif.get_next_data() object and the resulting map object is casted to a tuple
    nextDatas = tuple(map(lambda gif: gif.get_next_data(), gifs))
    # Stacking all the gif.get_next_data() objects horizontally
    new_image = np.hstack(nextDatas)
    finalGIF.append_data(new_image)

for gif in gifs:
    gif.close()

finalGIF.close()

In [ ]:
fig = px.scatter_3d(
    prevData,                   # df of Original Data
    x = prevData['t-SNE-comp-1'], 
    y = prevData['t-SNE-comp-2'], 
    z = prevData['t-SNE-comp-3'],              
    color='label',              # Assigning separate colors for different "label" value
    opacity=0.2,
    width=1200, height=800,     # Size of the output graph
)
fig.update_traces(              # changing the size of the markers
    marker=dict(
        size=4,
    ),
    selector=dict(mode='markers'))

fig.add_trace(                              # Now adding adding the centroid tsne results data
    go.Scatter3d(
        x = centroidData['t-SNE-comp-1'],
        y = centroidData['t-SNE-comp-2'],
        z = centroidData['t-SNE-comp-3'],
        mode='markers',
        marker=dict(
            size=12,
            color="black"
        ),
        name= 'Centroids',
        
    )
)

fig.show()

Image(open('CENTROIDS/Final Centroids.gif','rb').read()) # Showing the Generated Gif


In [ ]:
fig.write_html(f"CENTROIDS/Final Centroids 3D t-SNE k-{k} datapoints-{subset}.html")

In [ ]:
!zip -r /content/1910456.zip /content/CENTROIDS
from google.colab import files
files.download("/content/1910456.zip")

  adding: content/CENTROIDS/ (stored 0%)
  adding: content/CENTROIDS/centroid 14/ (stored 0%)
  adding: content/CENTROIDS/centroid 14/45.png (stored 0%)
  adding: content/CENTROIDS/centroid 14/19.png (stored 0%)
  adding: content/CENTROIDS/centroid 14/84.png (stored 0%)
  adding: content/CENTROIDS/centroid 14/28.png (stored 0%)
  adding: content/CENTROIDS/centroid 14/20.png (stored 0%)
  adding: content/CENTROIDS/centroid 14/31.png (stored 0%)
  adding: content/CENTROIDS/centroid 14/22.png (stored 0%)
  adding: content/CENTROIDS/centroid 14/39.png (stored 0%)
  adding: content/CENTROIDS/centroid 14/59.png (stored 0%)
  adding: content/CENTROIDS/centroid 14/2.png (stored 0%)
  adding: content/CENTROIDS/centroid 14/65.png (stored 0%)
  adding: content/CENTROIDS/centroid 14/46.png (stored 0%)
  adding: content/CENTROIDS/centroid 14/61.png (stored 0%)
  adding: content/CENTROIDS/centroid 14/48.png (stored 0%)
  adding: content/CENTROIDS/centroid 14/17.png (stored 0%)
  adding: content/CENT

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>